<a href="https://colab.research.google.com/github/ekdls02/ekdls2025/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D_%EA%B3%BC%EC%A0%9C4_%EB%87%8C_MRI_%EC%9D%B4%EB%AF%B8%EC%A7%80%EC%97%90%EC%84%9C_%EB%87%8C%EC%A2%85%EC%96%91_%EA%B2%80%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. 라이브러리 임포트 및 설정 (EfficientNetB0 클래스 명시적으로 임포트)
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
# EfficientNetB0 클래스를 직접 임포트하여 set_efficientnet_trainable 함수의 TypeError 해결
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras.backend import clear_session
from sklearn.model_selection import StratifiedKFold
import os

# ImageNet 통계값 (수동 정규화에 사용)
IMAGENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMAGENET_STDDEV = np.array([0.229, 0.224, 0.225], dtype=np.float32)

# 2. 경로 및 Seed 설정
TRAIN_PATH = "/content/train.npz"
TEST_PATH = "/content/test.npz"
SAMPLE_PATH = "/content/submission.csv"

MODEL_OUTPUT_PATH = "/content/new_submission/"
os.makedirs(MODEL_OUTPUT_PATH, exist_ok=True)

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 3. 데이터 로드 및 전처리 (수정: 레이블 매핑 및 수동 ImageNet 정규화)
train_data = np.load(TRAIN_PATH)
x_train_raw = train_data['x']
y_train_raw = train_data['y']

# 1채널 흑백 이미지를 3채널로 복사
x_train_temp = np.repeat(x_train_raw[..., np.newaxis], 3, axis=-1).astype('float32')

# 수동 ImageNet 표준 정규화 적용 (x / 255 - mean) / std
x_train = (x_train_temp / 255.0 - IMAGENET_MEAN) / IMAGENET_STDDEV

# 수정: 문자열 레이블을 이진 숫자로 변환 (ValueError 해결)
# y_train_raw의 내용이 ['normal', 'tumor', ...] 형태라고 가정
label_map = {'normal': 0, 'tumor': 1}
y_encoded = np.array([label_map[i] for i in y_train_raw])

# Input Shape 정의
input_shape = x_train.shape[1:]


# 4. Data Generator (수정: 증강 강도 약화)
BATCH_SIZE = 8 # Batch Size 축소
datagen = ImageDataGenerator(
    rotation_range=5,           # 5도로 축소
    width_shift_range=0.1,      # 0.1로 축소
    height_shift_range=0.1,     # 0.1로 축소
    horizontal_flip=True,
    vertical_flip=False,
    zoom_range=0.1,             # 0.1로 축소
    fill_mode='nearest'
)
def get_generator(x, y):
    return datagen.flow(x, y, batch_size=BATCH_SIZE, shuffle=True)


# 5. 모델 정의 (수정: set_efficientnet_trainable 안정화, Head 간소화/규제 강화)

def set_efficientnet_trainable(model, trainable_percent):
    """
    EfficientNetB0 내부 레이어만 찾아 trainable_percent 비율만큼 상위 레이어를 학습 가능하게 설정합니다.
    """
    efficientnet_base_model = None

    # 수정: 레이어 타입 대신, 이름에 'efficientnetb0'이 포함된 레이어를 찾습니다.
    # 이렇게 하면 isinstance() 관련 TypeError를 회피할 수 있습니다.
    for layer in model.layers:
        if 'efficientnetb0' in layer.name.lower():
            efficientnet_base_model = layer
            break

    if efficientnet_base_model is None:
        print("경고: EfficientNetB0 Base Model 레이어를 찾을 수 없습니다.")
        return

    # Base Model 내부의 학습 가능한 레이어만 필터링합니다.
    efficientnet_layers = [
        l for l in efficientnet_base_model.layers
        if 'block' in l.name or 'stem' in l.name or 'top' in l.name
    ]

    num_layers = len(efficientnet_layers)
    if num_layers == 0:
        print("경고: EfficientNetB0의 주요 블록 레이어를 찾을 수 없습니다.")
        return

    # 학습 가능한 레이어의 시작 인덱스 (상위 trainable_percent만 학습)
    trainable_start_index = int(num_layers * (1 - trainable_percent))

    layers_to_freeze = 0
    layers_to_train = 0

    for i, layer in enumerate(efficientnet_layers):
        if i < trainable_start_index:
            layer.trainable = False
            layers_to_freeze += 1
        else:
            layer.trainable = True
            layers_to_train += 1

    print(f"EfficientNetB0 내부: {layers_to_freeze}개 레이어 동결, {layers_to_train}개 레이어 학습 가능.")

def build_model(input_shape, base_model_trainable=False, trainable_percent=0.05):
    clear_session()

    inputs = Input(shape=input_shape)

    # EfficientNetB0 로드
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_tensor=inputs
        # 'trainable=False' 인수를 제거
    )

    # 수정: 모델 생성 직후, 명시적으로 동결
    # Head 학습 시에는 base_model.trainable=False가 되어야 합니다.
    # Fine-Tuning 시에는 아래 if 블록에서 다시 True가 됩니다.
    base_model.trainable = False

    # 3. Model Head 구성
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # Head 간소화 및 L2 규제 강화
    x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-3))(x)
    x = Dropout(0.5)(x)

    outputs = Dense(1, activation='sigmoid', dtype='float32')(x)

    model = Model(inputs, outputs)

    if base_model_trainable:
        # Fine-Tuning 시
        base_model.trainable = True # Base Model 전체를 학습 가능하도록 설정

        # Base Model 내부 레이어에 직접 접근하여 동결 및 학습 설정
        efficientnet_layers = [
            l for l in base_model.layers
            if 'block' in l.name or 'stem' in l.name or 'top' in l.name
        ]
        num_layers = len(efficientnet_layers)
        trainable_start_index = int(num_layers * (1 - trainable_percent))

        for i, layer in enumerate(efficientnet_layers):
            if i < trainable_start_index:
                layer.trainable = False
            else:
                layer.trainable = True

        # 출력 메시지
        print(f"EfficientNetB0 내부: {trainable_start_index}개 레이어 동결, {num_layers - trainable_start_index}개 레이어 학습 가능.")

    return model

# 6. Stratified K-Fold 학습 + Fine-Tuning (수정: EarlyStopping/LR 조정)
N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
ensemble_models = []
fold_val_accuracies = []

# 수정: 콜백 설정 (patience 축소)
early_stop_head = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True) # 8 -> 5
early_stop_ft = EarlyStopping(monitor='val_accuracy', mode='max', patience=8, restore_best_weights=True)  # 15 -> 8
reduce_lr_ft = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-7) # 5 -> 3

for fold, (train_idx, val_idx) in enumerate(skf.split(x_train, y_encoded)):
    print(f"Fold {fold+1}/{N_SPLITS}")
    X_tr, X_val = x_train[train_idx], x_train[val_idx]
    y_tr, y_val = y_encoded[train_idx], y_encoded[val_idx]

    ft_weights_path = os.path.join(MODEL_OUTPUT_PATH, f"fold{fold+1}_best_ft.keras")
    checkpoint_cb = ModelCheckpoint(ft_weights_path, monitor='val_accuracy',
                                    mode='max', save_best_only=True, verbose=1)

    # 1) Head 학습 (Base Model 동결)
    print("1. Head (분류기) 학습 시작")
    model_head = build_model(input_shape, base_model_trainable=False)

    model_head.compile(optimizer=SGD(learning_rate=1e-2, momentum=0.9),
                       loss='binary_crossentropy', metrics=['accuracy'])

    model_head.fit(get_generator(X_tr, y_tr), validation_data=(X_val, y_val),
                   epochs=50, callbacks=[early_stop_head], verbose=1)

    # 2) Fine-Tuning (수정: Base Model 상위 5% 학습 및 LR 하향)
    print("2. Fine-Tuning 시작")

    # 수정: Base Model의 상위 5%만 Fine-Tuning
    model_ft = build_model(input_shape, base_model_trainable=True, trainable_percent=0.05)
    model_ft.set_weights(model_head.get_weights())

    # 수정: Adam Learning Rate를 5e-6으로 하향
    model_ft.compile(optimizer=Adam(learning_rate=5e-6),
                      loss='binary_crossentropy', metrics=['accuracy'])

    model_ft.fit(get_generator(X_tr, y_tr), validation_data=(X_val, y_val),
                   epochs=150,
                   callbacks=[early_stop_ft, reduce_lr_ft, checkpoint_cb], verbose=1)

    # 3) 최종 평가: 최적의 Fine-Tuning 모델 로드 후 평가
    model_final = tf.keras.models.load_model(ft_weights_path)

    val_loss, val_acc = model_final.evaluate(X_val, y_val, verbose=0)
    fold_val_accuracies.append(val_acc)
    print(f"Fold {fold+1} 최종 검증 정확도: {val_acc:.5f}")

    ensemble_models.append(ft_weights_path)
    clear_session()


# 7. K-Fold 평균 검증 정확도
mean_acc = np.mean(fold_val_accuracies)
print(f"K-Fold 평균 검증 정확도 ({N_SPLITS} Folds): {mean_acc:.5f}")


# 8. Test 예측 및 Submission (앙상블 로직)
# Test 데이터 로드
test_data = np.load(TEST_PATH)
x_test_raw = test_data['x']

# Test 데이터도 학습 데이터와 동일하게 수동 ImageNet 정규화 적용
x_test_temp = np.repeat(x_test_raw[..., np.newaxis], 3, axis=-1).astype('float32')
x_test = (x_test_temp / 255.0 - IMAGENET_MEAN) / IMAGENET_STDDEV

test_preds_prob = []
# 각 Fold의 최적 모델로 예측 수행
for model_path in ensemble_models:
    model = tf.keras.models.load_model(model_path)
    preds_prob = model.predict(x_test, verbose=0)
    test_preds_prob.append(preds_prob)
    clear_session()

# 평균 예측 확률 계산 및 이진 분류
y_test_mean = np.mean(test_preds_prob, axis=0)
y_test_pred = (y_test_mean[:,0] >= 0.5).astype(int)

# 레이블 역변환
label_map_reverse = {0:'normal', 1:'tumor'}
y_test_str = np.array([label_map_reverse[i] for i in y_test_pred])

# Submission 파일 생성
sample = pd.read_csv(SAMPLE_PATH)
sample["result"] = y_test_str
submission_file_name = "submission_final_optimized.csv"
sample.to_csv(submission_file_name, index=False)
print(f"제출 파일 생성 완료: {submission_file_name}")

Fold 1/5
1. Head (분류기) 학습 시작
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


21/21 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - accuracy: 0.5529 - loss: 1.1239 - val_accuracy: 0.4878 - val_loss: 1.1120
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.4827 - loss: 1.1459 - val_accuracy: 0.7317 - val_loss: 1.0060
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.6292 - loss: 1.0610 - val_accuracy: 0.6098 - val_loss: 1.0431
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.6371 - loss: 1.1481 - val_accuracy: 0.6585 - val_loss: 1.0170
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.6445 - loss: 1.0325 - val_accuracy: 0.7073 - val_loss: 0.9853
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.6614 - loss: 1.0161 - val_accuracy: 0.8049 - val_loss: 0.9651
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 113ms/step - accuracy: 0.6720 - loss: 1.0847 - val_accuracy: 0.6098 - val_loss: 1.0590
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.6453 - loss: 1.0753 - val_accuracy: 0.7073 - val_loss: 0.

제출 파일 생성 완료: submission_final_optimized.csv
